In [ ]:
import sys
sys.path.append("/software/path/prefix/NvTK/")
import h5py, os, argparse, logging, time

import numpy as np
import pandas as pd

import torch
from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader

import NvTK
from NvTK import Trainer
from NvTK.Evaluator import calculate_correlation, calculate_pr, calculate_roc
from NvTK.Explainer import get_activate_W, meme_generate, save_activate_seqlets, calc_frequency_W

import matplotlib.pyplot as plt
from NvTK.Explainer import seq_logo, plot_seq_logo

#from NvTK import resnet18
from NvTK.Modules import BasicPredictor
# set_all_random_seed
NvTK.set_random_seed()
NvTK.set_torch_seed()
NvTK.set_torch_benchmark()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import scanpy as sc


sys.path.append("/file/path/prefix/")
n_tasks=50040 #cell num 
from ResNeXt_conv1_128_btnk_2dense import *
model = resnext34(num_classes = n_tasks)

# define criterion
criterion = nn.BCELoss().to(device)

# define optimizer
optimizer = Adam(model.parameters(), lr=1e-3, betas=(0.9, 0.999), eps=1e-08, weight_decay=0,)

# define trainer
trainer = Trainer(model, criterion, optimizer, device, 
                    patience=10, tasktype='binary_classification', metric_sample=100,
                    use_tensorbord=False)
## reload best model
model = trainer.load_best_model('./Log/best_model.pth')
model.eval()

In [ ]:
# Extrac FC layer weight -> cell embedding 

In [ ]:
cell_embedding = model.state_dict()['fc1.dense.weight']
cell_embedding.shape

In [ ]:
w = cell_embedding.cpu().numpy()
w.shape

In [ ]:
adata = sc.AnnData(w)
adata

In [ ]:
sc.set_figure_params(dpi=100, color_map = 'viridis_r')
sc.settings.verbosity = 1
sc.logging.print_header()

In [ ]:
anno = pd.read_table('./Female_mus_anno_5w.txt')
anno

In [ ]:
adata.obs['anno'] = anno.anno.values
adata.obs['lineage'] = anno.lineage.values
adata.obs['tissue'] = anno.Sample.values

In [ ]:
%%time
adata.obsm['projection'] = w
sc.pp.neighbors(adata, use_rep='projection')
sc.tl.tsne(adata)

In [ ]:
sc.pl.tsne(adata, color=['anno'], save=".w.anno")

In [ ]:
sc.pl.tsne(adata, color=['tissue'], save=".w.tissue")

In [ ]:
sc.pl.tsne(adata, color=['lineage'], save=".w.lineage")